## C卷测试流程

### C卷使用dockerID初步筛选
根据每个赛道各用户在B卷测试中的最高分作为C卷测试dockerID
>输出：`test_list_init.xlsx`

In [7]:
import os
import sys
sys.path.append('../..')
from connector import Connector

import pandas as pd

database = Connector()

init_record = {
    'pid': None,
    'pull': {'status': 'WAITING', 'cost': ''},
    'test': {'status': 'WAITING', 'cost': ''},
    'evaluate': {'status': 'WAITING', 'cost': ''},
    'upload': {'status': 'WAITING', 'cost': ''},
}

def get_competition_submits():
    script = f"SELECT * FROM submit WHERE (paperType='B') AND (competitionId IN ('2023031405', '2023031901', '2023031902', '2023031903'))"
    submits = database._db_fetch(script)
    return submits

def convert_time(time_use):
    time_use = float(time_use)
    h = int(time_use // 3600)
    m = int((time_use - h * 3600) // 60)
    s = int(time_use - h * 3600 - m * 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

def is_timeout(time_use, competitionId):
    time_use = float(time_use)
    if competitionId == '2023031405':
        if time_use > 3600 * 30:
            return True
        else:
            return False
    elif competitionId in ['2023031901', '2023031902', '2023031903']:
        if time_use > 3600 * 10:
            return True
        else:
            return False
    else:
        return False
    
def is_testing(submits):
    for s in submits['status']:
        if s == 'TESTING':
            return True
    return False

if __name__ == "__main__":
    excel_info = {
        'competitionName': [],
        'supervisorName': [],
        'school': [],
        'userName': [],
        'mobile': [],
        'dockerId': [],
        'score': [],
        'submitTime': [],
        'isTesting': [],
        'submitId': [],
    }
    
    submits = get_competition_submits()
    submits_df = pd.DataFrame(submits, columns=database.columns)
    for competition, competition_rows in submits_df.groupby('competitionId'):
        competitionName = database._get_value(table='competition', field='competitionName', logic='AND', competitionId=competition)[0][0]
        competition_rows.sort_values(by='score', ascending=True, inplace=True)
        for user, user_rows in competition_rows.groupby('submitterId'):
            user_info = database._get_value(table='user', field='userName, mobile, email, school, supervisorName', logic='AND', userId=user)[0]
            if user_info[0] in ['标杆算法1-IDM', '标杆算法2-Lattice', 'yangyh408']:
                continue
            excel_info['userName'].append(user_info[0])
            excel_info['mobile'].append(user_info[1])
            # excel_info['email'].append(user[2])
            excel_info['school'].append(user_info[3])
            excel_info['supervisorName'].append(user_info[4])
            
            excel_info['competitionName'].append(competitionName)
            
            if is_testing(user_rows):
                excel_info['isTesting'].append('TRUE')
            else:
                excel_info['isTesting'].append(None)
            
            user_rows.sort_values(by='score', ascending=False, inplace=True)
            highest_score = user_rows.iloc[0]
            excel_info['submitId'].append(highest_score['submitId'])
            excel_info['dockerId'].append(highest_score['dockerId'])
            excel_info['score'].append(highest_score['score'])
            excel_info['submitTime'].append(highest_score['submitTime'])
            
    excel_df = pd.DataFrame(excel_info)
    excel_df.to_excel('test_list_init.xlsx', index=False)

### 手动确认C卷测试内容
根据`test_list_init.xlsx`统计选手需求并汇总成`test_list.xlsx`

### 确认C卷测试情况
根据`test_list.xlsx`确定B卷信息登记无误并汇总C卷测试情况
> 输出：排行榜文件`leaderboard.xlsx`

In [6]:
import pandas as pd
import os
import sys
sys.path.append('../..')
from connector import Connector
import warnings
warnings.filterwarnings('ignore')

database = Connector()

df = pd.read_excel('./test_list.xlsx')
tests = df[['competitionName', '团队老师', '主用户名', '最终确认DockerID', 'B卷得分']].dropna(subset=['最终确认DockerID'])
# tests = df[['competitionName', '团队老师', '主用户名', '最终确认DockerID', 'B卷得分']].dropna()

C_competition_map = {
    '2023031405': '2023052701',
    '2023031901': '2023052702',
    '2023031902': '2023052703',
    '2023031903': '2023052704',
}
B_submitIds = []
C_scores = []
C_statuses = []
C_submitIds = []
for index, row in tests.iterrows():
    competitionId = database._get_value(table='competition', field='competitionId', logic='AND', competitionName=row['competitionName'])[0][0]
    userId = database._get_value(table='user', field='userId', logic='AND', userName=row['主用户名'])[0][0]
    try:
        script = f"SELECT score, submitId FROM submit WHERE competitionId='{competitionId}' AND submitterId='{userId}' AND dockerId='{row['最终确认DockerID'].strip()}' ORDER BY score DESC LIMIT 1"
        B_score, submitId = database._db_fetch(script)[0]
        if row['B卷得分'] != B_score:
            print(competitionId, userId, row['最终确认DockerID'], row['B卷得分'], B_score)
        B_submitIds.append(submitId)
    except:
        print(competitionId, userId, row['主用户名'], row['最终确认DockerID'])
    
    try:
        script = f"SELECT status, score, submitId FROM submit WHERE competitionId='{C_competition_map[competitionId]}' AND dockerId='{row['最终确认DockerID'].strip()}' AND month(submitTime)>=6 ORDER BY score DESC LIMIT 1"
        C_stauts, C_score, submitId = database._db_fetch(script)[0]
        C_statuses.append(C_stauts)
        C_submitIds.append(submitId)
        if not C_score and  C_stauts not in ['QUEUING', 'WAITING', 'TESTING']:
            C_scores.append(0)
        else:
            C_scores.append(C_score)
    except:
        C_scores.append(None)
        C_submitIds.append(None)
        C_statuses.append('未测试')
    
tests['B卷排名'] = tests.groupby('competitionName')['B卷得分'].rank(method='min', ascending=False)
tests['B卷submitId'] = B_submitIds
tests['C卷测试状态'] = C_statuses
tests['C卷得分'] = C_scores
tests['C卷排名'] = tests.groupby('competitionName')['C卷得分'].rank(method='min', ascending=False)
tests['C卷submitId'] = C_submitIds

tests['总得分'] = tests['B卷得分'] * 0.7 + tests['C卷得分'] * 0.3
tests['总排名'] = tests.groupby('competitionName')['总得分'].rank(method='min', ascending=False)

tests.sort_values(by=['competitionName', 'B卷得分'], ascending=False, inplace=True)

In [7]:
# 创建一个ExcelWriter对象，并将每个分组的数据写入到不同的Excel工作表中
writer = pd.ExcelWriter('leaderboard.xlsx')
for name, group in tests.groupby('competitionName'):
    group.assign()
    group.to_excel(writer, sheet_name=str(name), index=False)
    
# 保存并关闭ExcelWriter对象
writer.close()

### 根据B卷筛选得分修改排行榜最高分

In [2]:
import sys
sys.path.append('../..')
from connector import Connector
import pandas as pd

database = Connector()
leaderboard = pd.read_excel('leaderboard_final.xlsx', sheet_name=None)

for competition, competition_rows in leaderboard.items():
    competitionId = database._get_value(table='competition', field='competitionId', logic='AND', competitionName=competition)[0][0]
    for index, row in competition_rows.iterrows():
        userId = database._get_value(table='user', field='userId', logic='AND', userName=row['主用户名'])[0][0]
        # 确认排行榜筛选的B卷得分为该规控器的最高分
        scirpt = f"SELECT score, submitId FROM submit WHERE dockerId='{row['最终确认DockerID']}' AND paperType='B' AND submitterId={userId} And competitionId={competitionId} Order BY score DESC LIMIT 1"
        score, submitId = database._db_fetch(scirpt)[0]
        if score != row['B卷得分'] or submitId != row['B卷submitId']:
            print(f"信息有误：{competition}->{row['主用户名']} {row['最终确认DockerID']}")
        # 查看该用户提交中比B卷确认的得分高的提交
        scirpt = f"SELECT * FROM submit WHERE paperType='B' AND submitterId={userId} And competitionId={competitionId} AND status='SUCCESS'"
        user_submits = database._db_fetch(scirpt)
        user_df = pd.DataFrame(user_submits, columns=database._get_col_name('submit'))
        user_df.dropna(subset=['score'], inplace=True)
        higher = user_df[user_df['score']>row['B卷得分']]
        for index, higher_row in higher.iterrows():
            # print(f"select score: {row['B卷得分']} --> best score: {higher[['score', 'submitId']]}")
            print(higher_row['submitId'], higher_row['score'], row['B卷得分'])
            database.update(table = 'submit', info = {'score': None}, logic = 'AND', submitId = higher_row['submitId'])
            
"""被替换的submitId，第二列为该submit得分
s_20230531235351_20230425082036 92.064 91.9783
s_20230529170159_20230322113454 68.1016 67.406
s_20230531103808_20230319065149 93.4214 93.4213
s_20230528221639_20230423094805 78.882 78.864
s_20230529134430_20230322113454 76.7321 76.6421
s_20230525191327_20230430104525 96.6997 96.4773
"""

'被替换的submitId，第二列为该submit得分\ns_20230531235351_20230425082036 92.064 91.9783\ns_20230529170159_20230322113454 68.1016 67.406\ns_20230531103808_20230319065149 93.4214 93.4213\ns_20230528221639_20230423094805 78.882 78.864\ns_20230529134430_20230322113454 76.7321 76.6421\ns_20230525191327_20230430104525 96.6997 96.4773\n'

### C卷场景筛选
1. 下载选手输出文件
2. 修改文件夹中成绩记录文件`score.csv`
3. 删除文件夹中无用的场景输出文件
4. 重命名文件夹名称
5. 上传到又拍云
6. 将新的提交信息与用户指定主用户名绑定
> 输入：`C_select_scenes.xlsx`

#### 下载结果

In [ ]:
import os
import urllib.request
import zipfile
import pandas as pd
from tqdm import tqdm

base_dir = "/home/ubuntu/onsite-test-server/temp"

def download_result(submitId, url):
    filename = url.split('/')[-1]
    filepath = os.path.join(base_dir, filename)
    extract_dir = os.path.join(base_dir, submitId)
    if not os.path.exists(extract_dir):
        if not os.path.exists(filepath):
            urllib.request.urlretrieve(url, filepath)
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        os.remove(filepath)
    
leaderboard = pd.read_excel('leaderboard.xlsx', sheet_name=None)
for competition, rows in leaderboard.items():
    with tqdm(total=len(rows)) as _tqdm:
        _tqdm.set_description(f"{competition}")    
        for index, row in rows.iterrows():
            _tqdm.update(1)
            resultLink = database._get_value(table='submit', field='resultLink', logic='AND', submitId=row['C卷submitId'])[0][0]
            if resultLink:
                download_result(row['C卷submitId'], resultLink)
                # print(f"{row['C卷submitId']}下载完成!")
            else:
                print(f"{row['C卷submitId']}未找到下载链接!")

#### 获得筛选后的得分

In [9]:
import os
import pandas as pd
import sys
sys.path.append('../..')
from connector import Connector

base_dir = "/home/ubuntu/onsite-test-server/temp"

database = Connector()
writer = pd.ExcelWriter('leaderboard_final.xlsx')

def cal_select_score(submitId, scenes):
    columns = ['Scenario', 'Safety', 'Efficiency', 'Comfort', 'Total']
    score_path = os.path.join(base_dir, submitId, 'score.csv')
    
    if os.path.exists(score_path):
        scores = pd.read_csv(score_path)[columns]
        scores = scores[scores['Scenario'].isin(scenes)]
        return scores['Total'].sum()/len(scenes)
    else:
        print(f"{submitId}未找到score.csv文件!")
        return 0

select_scenes = {}
select_scenes_df = pd.read_excel('C_select_scenes.xlsx')
for competitionName, scenes in select_scenes_df.items():
    select_scenes[competitionName] = scenes.dropna().tolist()
    
leaderboard = pd.read_excel('leaderboard.xlsx', sheet_name=None)
for competition, rows in leaderboard.items(): 
    C_scores = []
    schools = []
    for index, row in rows.iterrows():
        C_scores.append(cal_select_score(row['C卷submitId'], select_scenes[competition]))
        schools.append(database._get_value(table='user', field='school', logic='AND', supervisorName=row['团队老师'])[0][0])
    rows['C卷得分'] = C_scores
    rows['C卷排名'] = rows['C卷得分'].rank(method='min', ascending=False)

    rows['总得分'] = rows['B卷得分'] * 0.7 + rows['C卷得分'] * 0.3
    rows['总排名'] = rows['总得分'].rank(method='min', ascending=False)
    
    rows['学校'] = schools

    rows.sort_values(by=['总得分'], ascending=False, inplace=True)
    
    rows = rows[['主用户名', '学校', '团队老师', '最终确认DockerID', 'B卷得分', 'B卷排名', 'B卷submitId', 'C卷得分', 'C卷排名', 'C卷submitId', '总得分', '总排名']]
    rows.round(4).to_excel(writer, sheet_name=competition, index=False)

writer.close()

s_20230603185728_20230316121333未找到score.csv文件!


In [2]:
import os
import datetime
import pandas as pd

base_dir = "/home/ubuntu/onsite-test-server/temp"
    
def filter_files(submitId, new_submitId, select_scene):
    submit_dir = os.path.join(base_dir, submitId)
    new_submit_dir = os.path.join(base_dir, new_submitId)
    score_path = os.path.join(submit_dir, 'score.csv')
    
    columns = ['Scenario', 'Safety', 'Efficiency', 'Comfort', 'Total']
    final_score = 0
    
    if os.path.exists(score_path):
        scores = pd.read_csv(score_path)[columns]
        select_scores = pd.DataFrame(columns=columns)
        for scene in select_scene:
            row = scores[scores['Scenario'] == scene]
            if not row.empty:
                select_scores = pd.concat([select_scores, row], ignore_index=True)
            else:
                select_scores = pd.concat([select_scores, pd.DataFrame([[scene,0,0,0,0]], columns=columns)], ignore_index=True)
        mean_row = select_scores[columns[1:]].mean(axis=0).tolist()
        mean_row.insert(0, 'Mean')
        final_score = mean_row[-1]
        select_scores = pd.concat([select_scores, pd.DataFrame([mean_row], columns=columns)], ignore_index=True)
        select_scores.round(4).to_csv(score_path, index=False)
        
    for file_name in os.listdir(submit_dir):
        if file_name.endswith('.csv') and not file_name.startswith('score'):
            if file_name[:-11] not in select_scene:
                os.remove(os.path.join(submit_dir, file_name))
                
    os.rename(submit_dir, new_submit_dir)
    return final_score

def generate_C_submition(submitId, userName):
    C_competition_map = {
        '2023052701': '2023031405',
        '2023052702': '2023031901',
        '2023052703': '2023031902',
        '2023052704': '2023031903',
    }
    res = database._get_value(table='submit', field='*', logic='AND', submitId=submitId)
    res = dict(zip(database.columns, res[0]))
    res['competitionId'] = C_competition_map[res['competitionId']]
    
    submitterId = database._get_value(table='user', field='userId', logic='AND', userName=userName)[0][0]
    res['submitterId'] = submitterId
    res['submitId'] = res['submitId'][:17] + submitterId
    res['paperType'] = 'C'
    res['resultLink'] = None
    res['submitTime'] = datetime.datetime.strftime(res['submitTime'], "%Y-%m-%d %H:%M:%S")
    res['testTime'] = datetime.datetime.strftime(res['testTime'], "%Y-%m-%d %H:%M:%S")
    return res

In [4]:
# 构造选手C卷提交及结果
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append('../..')
from connector import Connector
from uploader import Uploader

database = Connector()
base_dir = "/home/ubuntu/onsite-test-server/temp"

select_scenes = {}
select_scenes_df = pd.read_excel('C_select_scenes.xlsx')
for competitionName, scenes in select_scenes_df.items():
    select_scenes[competitionName] = scenes.dropna().tolist()

leaderboard = pd.read_excel('leaderboard_final.xlsx', sheet_name=None)
for competition, rows in leaderboard.items(): 
    for index, row in rows.iterrows():
        resultLink = database._get_value(table='submit', field='resultLink', logic='AND', submitId=row['C卷submitId'])[0][0]
        if resultLink:
            new_submit_info = generate_C_submition(row['C卷submitId'], row['主用户名'])
            res = database._get_value(table='submit', field='*', logic='AND', submitId=new_submit_info['submitId'])
            if res:
                # print(f"{row['C卷submitId']} --> {new_submit_info['submitId']}")
                continue
            download_result(row['C卷submitId'], resultLink)
            score = filter_files(row['C卷submitId'], new_submit_info['submitId'], select_scenes[competition])
            new_submit_info['score'] = score
            uploader = Uploader(base_dir, new_submit_info)
            try:
                uploader._upload_file()
                new_submit_info['resultLink'] = uploader.result_link
                print(f"{competition} {row['主用户名']} {score}<-cal-excel->{row['C卷得分']} {new_submit_info}")
            except Exception as e:
                print(f"UPLOAD ERROR: {row['主用户名']} ({row['C卷submitId']}->{new_submit_info['submitId']})")
            database.insert(table = 'submit', info = new_submit_info)
        else:
            print(f"{row['C卷submitId']}未找到下载链接!")

### 构建GIF图在数据库sceneSubmit表中的提交

In [26]:
import os
import urllib.request
import zipfile
import time
import sys
sys.path.append('../..')
from connector import Connector
import pandas as pd

database = Connector()
base_dir = "/home/ubuntu/onsite-test-server/temp"

def download_result(submitId, url):
    filename = url.split('/')[-1]
    filepath = os.path.join(base_dir, filename)
    extract_dir = os.path.join(base_dir, submitId)
    if not os.path.exists(extract_dir):
        if not os.path.exists(filepath):
            urllib.request.urlretrieve(url, filepath)
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        os.remove(filepath)

scene_submit_info = {
    'sceneName': "",
    'userName': "",
    'score': "",
    'submitTime': "",
    'submitUr': "",
}

leaderboard = pd.read_excel('leaderboard_final.xlsx', sheet_name=None)
for competition, rows in leaderboard.items():
    if '综合赛' in competition:
        continue
    competitionId = database._get_value(table='competition', field='competitionId', logic='AND', competitionName=competition)[0][0]
    for userName in rows.head(3)['主用户名'].tolist():
        userId = database._get_value(table='user', field='userId', logic='AND', userName=userName)[0][0]
        submit_info = database._get_value(table='submit', field='*', logic='AND', submitterId=userId, competitionId=competitionId, paperType='C')[0]
        submit_info = dict(zip(database.columns, submit_info))
        download_result(submit_info['submitId'], submit_info['resultLink'])
        score_path = os.path.join(base_dir, submit_info['submitId'], 'score.csv')
        for index, row in pd.read_csv(score_path).iterrows():
            if row['Scenario'] != 'Mean':
                if os.path.exists(os.path.join(base_dir, submit_info['submitId'], f"{row['Scenario']}_result.csv")):
                    new_submit_info = {
                        'sceneName': row['Scenario'],
                        'userName': userName,
                        'score': row['Total'],
                        'submitTime': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'submitUrl': f"https://resource.onsite.run/new_onsite_media/GIFS/{submit_info['submitId']}/{submit_info['submitId']}-{row['Scenario']}_exam.gif",
                    }
                    # print(new_submit_info['submitUrl'])
                    database.insert(table='sceneSubmit', info=new_submit_info)
    

### 发送邮件

In [5]:
import smtplib
from email.mime.text import MIMEText
from email.utils import formataddr

def send_email(receiver_name, receiver_email, sub, msg):

    # 发送方邮箱地址和密码
    sender_email = 'onsite_challenge@163.com'
    sender_password = 'PDCJWBNYKSEBOMSW'

    # 构造邮件内容
    message = MIMEText(msg, "plain", "utf-8")
    message['From'] = formataddr(["OnSite第一届算法挑战赛竞赛组委会", sender_email])
    message['To'] = formataddr([receiver_name, receiver_email])
    message['Subject'] = sub

    # 连接到 SMTP 服务器并发送邮件
    with smtplib.SMTP_SSL("smtp.163.com", 465) as server:
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, [receiver_email], message.as_string())
        
send_email('yangyh408', '895258058@qq.com', "Python 测试邮件", "这是一封来自 Python 的测试邮件。")

In [14]:
import sys
sys.path.append('../..')
from connector import Connector
import pandas as pd

database = Connector()

df = pd.read_excel('leaderboard_final.xlsx', sheet_name=None)

res = []
for competition, competition_rows in df.items():
    for index, row in competition_rows.head(4).iterrows():
        user_info = database._get_value(table='user', field='*', logic='AND', userName=row['主用户名'])[0]
        user_info = dict(zip(database._get_col_name('user'), user_info))
        res.append([competition, row['学校'], row['团队老师'], row['主用户名'], user_info['email'], user_info['mobile']])
pd.DataFrame(res, columns=['比赛名称', '学校', '团队老师', '主用户名', '邮箱', '手机号']).to_csv('leader_info.csv', index=False, encoding='gbk')